In [2]:
import pandas as pd
import numpy as np
import seaborn as sn

In [9]:
df_cs

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Software Engineer,$60K-$63K (Glassdoor est.),Job Description\n\nCSI’s Payments Software Eng...,4.0,Computer Services\n4.0,"Valparaiso, IN","Paducah, KY",1001 to 5000 employees,1965,Company - Public,Financial Transaction Processing,Finance,$100 to $500 million (USD),-1
1,1,Software Engineer,$60K-$63K (Glassdoor est.),We are AAM. We have the POWER to move the worl...,3.3,American Axle & Manufacturing\n3.3,"Detroit, MI","Detroit, MI",10000+ employees,1994,Company - Public,Transportation Equipment Manufacturing,Manufacturing,$5 to $10 billion (USD),-1
2,2,Software Engineer,$60K-$63K (Glassdoor est.),Preferred Qualifications\nA strong foundation ...,3.9,Quicken Loans\n3.9,"Detroit, MI","Detroit, MI",10000+ employees,1985,Company - Private,Lending,Finance,$10+ billion (USD),"Citi, Bank of America, Wells Fargo"
3,3,Test Engineer,$60K-$63K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nBecom...",4.1,ManTech International Corporation\n4.1,"Clarksburg, WV","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
4,4,Lead Embedded Software Engineer - Wearables,$60K-$63K (Glassdoor est.),Job Description\n\n\nBose Corporation’s Consum...,3.6,Bose\n3.6,"Framingham, MA","Framingham, MA",5001 to 10000 employees,1964,Company - Private,Consumer Products Manufacturing,Manufacturing,$2 to $5 billion (USD),-1
5,5,Software Engineer II,$60K-$63K (Glassdoor est.),Software Engineer II\n\nThis Software Engineer...,3.0,Kryterion Inc\n3.0,"Phoenix, AZ","Phoenix, AZ",51 to 200 employees,2001,Company - Private,IT Services,Information Technology,$10 to $25 million (USD),"Pearson VUE, Prometric, PSI Services"
6,6,Software Engineer,$60K-$63K (Glassdoor est.),"Who We Are:\nBectran, Inc. was founded in 2010...",2.7,Bectran\n2.7,"Schaumburg, IL","Schaumburg, IL",51 to 200 employees,2010,Company - Private,Computer Hardware & Software,Information Technology,$10 to $25 million (USD),-1
7,7,Senior Software Engineer,$60K-$63K (Glassdoor est.),"Buildium is looking for smart, driven, enthusi...",4.8,Buildium\n4.8,"Boston, MA","Boston, MA",51 to 200 employees,2004,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
8,8,"Jr. Software Engineer (JAVA, C/C++)",$60K-$63K (Glassdoor est.),Overview\n\n\nPeraton is seeking a highly moti...,3.4,Peraton\n3.4,"Aurora, CO","Herndon, VA",1001 to 5000 employees,2017,Company - Private,Aerospace & Defense,Aerospace & Defense,$1 to $2 billion (USD),-1
9,9,"Software Engineer - Remote, USA",$60K-$63K (Glassdoor est.),Are you passionate about building beautiful Us...,4.1,Guidewire Software\n4.1,Remote,"San Mateo, CA",1001 to 5000 employees,2001,Company - Public,Computer Hardware & Software,Information Technology,$500 million to $1 billion (USD),"Oracle, Accenture, Duck Creek Technologies"


In [11]:
titles = df_cs.get('Job Title')
titles

0                                      Software Engineer
1                                      Software Engineer
2                                      Software Engineer
3                                          Test Engineer
4            Lead Embedded Software Engineer - Wearables
5                                   Software Engineer II
6                                      Software Engineer
7                               Senior Software Engineer
8                    Jr. Software Engineer (JAVA, C/C++)
9                        Software Engineer - Remote, USA
10                                     Software Engineer
11                                     Software Engineer
12                                     Software Engineer
13                         Software Developer (Back-End)
14                                     Software Engineer
15                          Full-Stack Software Engineer
16     L1 Modem Verification and Release Software Eng...
17                  Software En

In [16]:
def standardize_title(title):
    
    title = title.lower()
    title = title.strip()
    
    if 'software engineer' in title.lower():
        return 'software engineer'
    if 'senior' or 'Sr.' in title.lower():
        return 'senior software engineer'
    

In [17]:
df_cs['title_simp'] = df_cs['Job Title'].apply(standardize_title)

In [18]:
df_cs

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,title_simp
0,0,Software Engineer,$60K-$63K (Glassdoor est.),Job Description\n\nCSI’s Payments Software Eng...,4.0,Computer Services\n4.0,"Valparaiso, IN","Paducah, KY",1001 to 5000 employees,1965,Company - Public,Financial Transaction Processing,Finance,$100 to $500 million (USD),-1,software engineer
1,1,Software Engineer,$60K-$63K (Glassdoor est.),We are AAM. We have the POWER to move the worl...,3.3,American Axle & Manufacturing\n3.3,"Detroit, MI","Detroit, MI",10000+ employees,1994,Company - Public,Transportation Equipment Manufacturing,Manufacturing,$5 to $10 billion (USD),-1,software engineer
2,2,Software Engineer,$60K-$63K (Glassdoor est.),Preferred Qualifications\nA strong foundation ...,3.9,Quicken Loans\n3.9,"Detroit, MI","Detroit, MI",10000+ employees,1985,Company - Private,Lending,Finance,$10+ billion (USD),"Citi, Bank of America, Wells Fargo",software engineer
3,3,Test Engineer,$60K-$63K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nBecom...",4.1,ManTech International Corporation\n4.1,"Clarksburg, WV","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,senior software engineer
4,4,Lead Embedded Software Engineer - Wearables,$60K-$63K (Glassdoor est.),Job Description\n\n\nBose Corporation’s Consum...,3.6,Bose\n3.6,"Framingham, MA","Framingham, MA",5001 to 10000 employees,1964,Company - Private,Consumer Products Manufacturing,Manufacturing,$2 to $5 billion (USD),-1,software engineer
5,5,Software Engineer II,$60K-$63K (Glassdoor est.),Software Engineer II\n\nThis Software Engineer...,3.0,Kryterion Inc\n3.0,"Phoenix, AZ","Phoenix, AZ",51 to 200 employees,2001,Company - Private,IT Services,Information Technology,$10 to $25 million (USD),"Pearson VUE, Prometric, PSI Services",software engineer
6,6,Software Engineer,$60K-$63K (Glassdoor est.),"Who We Are:\nBectran, Inc. was founded in 2010...",2.7,Bectran\n2.7,"Schaumburg, IL","Schaumburg, IL",51 to 200 employees,2010,Company - Private,Computer Hardware & Software,Information Technology,$10 to $25 million (USD),-1,software engineer
7,7,Senior Software Engineer,$60K-$63K (Glassdoor est.),"Buildium is looking for smart, driven, enthusi...",4.8,Buildium\n4.8,"Boston, MA","Boston, MA",51 to 200 employees,2004,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1,software engineer
8,8,"Jr. Software Engineer (JAVA, C/C++)",$60K-$63K (Glassdoor est.),Overview\n\n\nPeraton is seeking a highly moti...,3.4,Peraton\n3.4,"Aurora, CO","Herndon, VA",1001 to 5000 employees,2017,Company - Private,Aerospace & Defense,Aerospace & Defense,$1 to $2 billion (USD),-1,software engineer
9,9,"Software Engineer - Remote, USA",$60K-$63K (Glassdoor est.),Are you passionate about building beautiful Us...,4.1,Guidewire Software\n4.1,Remote,"San Mateo, CA",1001 to 5000 employees,2001,Company - Public,Computer Hardware & Software,Information Technology,$500 million to $1 billion (USD),"Oracle, Accenture, Duck Creek Technologies",software engineer


In [15]:
df_cs['Job Title'].unique()

array(['Software Engineer', 'Test Engineer',
       'Lead Embedded Software Engineer - Wearables',
       'Software Engineer II', 'Senior Software Engineer',
       'Jr. Software Engineer (JAVA, C/C++)',
       'Software Engineer - Remote, USA', 'Software Developer (Back-End)',
       'Full-Stack Software Engineer',
       'L1 Modem Verification and Release Software Engineer',
       'Software Engineer - Web Applications',
       'Controls and Software Engineer', 'Web Application Developer',
       'Embedded Software Engineer', 'Software Development Engineer',
       'Software Integration Engineer', 'Software Support Engineer',
       'Transportation Design Engineer', 'Application Developer',
       'Software Engineer - Server Applications',
       'Manager, Software Engineering', 'Software Engineer - Membership',
       'Mid-Level Software Systems Engineer', 'Full Stack Engineer',
       'Full Stack Software Engineer', 'Backend Engineer',
       'Software Engineer Associate', 'Service